In [1]:
import dask
import matplotlib.pyplot as plt
import numpy as np
from dask.distributed import Client, LocalCluster
import xarray as xr
import os
import pyextremes as pe
import sys
import dask.array as da

In [2]:
chunks = {'lat': 10, 'lon':10}
storage_directory = '/home/davidbyrne/disks/disk-2/inputs/'
filename_list = ['CMIP6_regridded_data_r288x192_maximum_daily_wetbulb_temperature_thresholds_2022_ACCESS-CM2_wetbulb_1deg_2005_2025.nc',
                 'CMIP6_regridded_data_r288x192_maximum_daily_wetbulb_temperature_thresholds_2022_ACCESS-ESM1-5_wetbulb_1deg_2004_2024.nc',
                 'CMIP6_regridded_data_r288x192_maximum_daily_wetbulb_temperature_thresholds_2022_CMCC-ESM2_wetbulb_1deg_1994_2014.nc']

In [3]:
#cluster = LocalCluster(port=44177)
client = Client("tcp://127.0.0.1:33501")
client
#client.get_versions()
#client = Client(n_workers=4, threads_per_worker=1)

<Client: 'tcp://127.0.0.1:33501' processes=6 threads=30, memory=117.92 GiB>

In [4]:
dataset = xr.open_dataset(os.path.join(storage_directory, filename_list[0]), chunks=chunks)
variable = dataset['Twb'].data

FileNotFoundError: [Errno 2] No such file or directory: b'/home/davidbyrne/disks/disk-2/inputs/CMIP6_regridded_data_r288x192_maximum_daily_wetbulb_temperature_thresholds_2022_ACCESS-CM2_wetbulb_1deg_2005_2025.nc'

### 1. dask.array.max()

In [6]:
%%time
# 1. USE DA.MAX
variable = da.random.random((1000000, 100, 100))
max = da.max(variable, axis=0).compute()

CPU times: user 130 ms, sys: 0 ns, total: 130 ms
Wall time: 5.82 s


### 2. Delayed list

In [ ]:
%%time
# 2. Via delayed objects
chunks = variable.to_delayed().ravel()

@dask.delayed
def test_function(chunk):
    mm = da.max(chunk, axis=0)
    return mm

results = [test_function(b) for b in chunks] 
#results = [rr.compute() for rr in results]
results2 = dask.compute(*results)


### 3. Map_blocks()

In [ ]:
def test_function(chunk):
    return da.max(chunk, 0)

results = da.map_blocks(test_function, variable, drop_axis=0, )
results = results.compute()

In [ ]:
plt.imshow(results)

### 4. Multiple max for multiple chunks in time

In [12]:
dataset_list = [ xr.open_dataset( os.path.join(storage_directory, ff), chunks=chunks ) for ff in filename_list] 
dataset_concat = xr.concat(dataset_list, dim='time')
variable = dataset_concat.chunk({'time':-1, 'lon': 10, 'lat':10})
variable.to_zarr('/home/davidbyrne/disks/disk-2/outputs/test')

In [14]:
testz

,Array,Chunk
Bytes,4.74 GiB,8.78 MiB
Shape,"(23005, 192, 288)","(23005, 10, 10)"
Count,581 Tasks,580 Chunks
Type,float32,numpy.ndarray


In [4]:
%%time
testz = xr.open_zarr('/home/davidbyrne/disks/disk-2/outputs/test')['Twb'].data

CPU times: user 15.2 ms, sys: 704 µs, total: 15.9 ms
Wall time: 46.3 ms


In [17]:
%%time 
max1 = da.max(testz, axis=0)
max1 = max1.compute()

CPU times: user 258 ms, sys: 4.26 ms, total: 262 ms
Wall time: 5.28 s


### 5. Zarr: Delayed list

In [39]:
%%time
# 2. Via delayed objects
chunks = testz.to_delayed().ravel()

@dask.delayed
def test_function(chunk, index):
    sh = chunk.shape
    mm = da.zeros((sh[1], sh[2]))
    for ii in range(sh[1]):
        for jj in range(sh[2]):
            tmp = chunk[:,ii,jj]
            mm_tmp = da.max(tmp)
            mm[ii,jj] = mm_tmp
    print(index)
    return mm

results = [test_function(b, index) for index, b in enumerate(chunks)] 
results2 = client.compute(results)

CPU times: user 471 ms, sys: 26.3 ms, total: 497 ms
Wall time: 496 ms


In [41]:
results3 = [da.from_delayed(rr, shape=(10,10), dtype=float) for rr in results]

In [ ]:
results4 = dask.compute(results3)

AttributeError: 'list' object has no attribute 'shape'

### 6. Zarr with map_blocks()

In [5]:
def test_function(chunk):
    return da.max(chunk, 0)

results = da.map_blocks(test_function, testz, drop_axis=0, )
results = results.compute()

In [6]:
results

dask.array<concatenate, shape=(192, 288), dtype=float32, chunksize=(10, 10), chunktype=numpy.ndarray>